<a href="https://colab.research.google.com/github/ShreyaJaiswal1604/Arthritis_Image_Classification/blob/main/notebooks/Arthritis_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
Step 1: Import Libraries

---


In [5]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, DenseNet121, VGG16


In [6]:
!git clone https://github.com/ShreyaJaiswal1604/Arthritis_Image_Classification

fatal: destination path 'Arthritis_Image_Classification' already exists and is not an empty directory.


---

Section 2: Define Paths and Parameters

---

In [7]:
# Define data directories
base_dir = "./Arthritis_Image_Classification/dataset/Original_Knee_Osteoarthritis_Dataset"
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')
batch_size = 32

---

Step 3: Create ImageDataGenerator for Preprocessing

---


In [8]:
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)


---

Step 4: Generate Datasets

---

In [9]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)


Found 2459 images belonging to 2 classes.
Found 355 images belonging to 2 classes.
Found 690 images belonging to 2 classes.


---
Define a Function to Create and Train a Model

---

In [10]:
# Function to create and train a model
def train_model(base_model, model_name):
    # Create a sequential model
    model = tf.keras.models.Sequential()

    # Add the base model
    model.add(base_model)

    # Global average pooling layer
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # Dense layer with ReLU activation
    model.add(tf.keras.layers.Dense(256, activation='relu'))

    # Dropout layer to prevent overfitting
    model.add(tf.keras.layers.Dropout(0.5))

    # Output layer with sigmoid activation for binary classification
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"{model_name} Test Accuracy: {test_accuracy}")

    # Plot training history
    plt.figure(figsize=(12, 4))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

    return model, history


In [11]:
# Train ResNet model
resnet_model, resnet_history = train_model(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)), 'ResNet')


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
 4/76 [>.............................] - ETA: 28:38 - loss: 0.5456 - accuracy: 0.8672

KeyboardInterrupt: 

In [ ]:
# Train DenseNet model
densenet_model, densenet_history = train_model(DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3)), 'DenseNet')


In [ ]:
# Train VGG16 model
vgg16_model, vgg16_history = train_model(VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)), 'VGG16')